## **Advanced Data Visualization using hdf5 datafiles from Institut Néel**
- Version: 2.0.2 <br>
- Author: William Rigaut <br>
- Date: 28.10.2025  <br>
- Package developped at Institut Néel to visualize and manipulate the data with python commands. A tool (DaHU) to quickly visualize the data is already available at https://github.com/Combinatorials-Neel/DaHU. This Notebook will allow you to customize plots with much less restrictions but needs basic python knowledge.

### **1. Introduction**

<p>An <b>HDF5</b> datafile compiled from high-throughput experiments at Institut Néel contains different measurement scans for each point in a 2D-grid over a thin film deposited on a 100mm substrate. This 2D-grid contains a point every 5mm centered around the center of the wafer itself, by considering the circular shape of the substrate one datafile contains 249 scans for each main characterization technique.</p>

<p>At each position, four main characterization techniques can be performed: <p>

- <b>Energy Dispersive X-ray spectroscopy (EDX)</b> where element composition can be determined using a scanning electron microscope.

- <b>Scanning Magneto Optics Kerr Effect (MOKE)</b> where the coercive field on the surface of a magnetic thin film can be obtained by a fast optical measurement under a pulsed magnetic field.

- <b>X-Ray Diffraction (XRD)</b> where structural properties can be determined (Phases and lattice parameters) using either a automated laboratory diffractometer or synchrotron radiation.

- <b>Profilometry (DEKTAK)</b>  is also used to estimate magnetic film thickness at every position with a calibration experiment using a patterned substrate. This technique is new and it has not been applied to older datasets. Profilometry data will be added in a futur version of the package and is currently not supported.

The HDF5 file structure is defined using a standard very similar to the <b>NeXuS hdf5 file format</b> used in synchrotron facilities (https://www.nexusformat.org/).

<span style="color: red;">Disclamer: Older datasets might not be entirely readable by the current version of the package.</span>

### **2. Read and display data from hdf5 file using a GUI**

A first overview of the HDF5 datafile can be done using a graphical user interface (GUI) program such as:
- `pyMca` http://www.silx.org/doc/PyMca/dev/install.html
- `hdf5view` https://pypi.org/project/hdf5view/

<p>Without GUI, the python module <b>h5py</b>, which is actually used by the reader can also be considered.</p>

<p>Here is an example with the PyMca software:</p>

![HDF5 file display using PyMca software](examples/Figure_1-PyMca.png)

Inside this hdf5 datafile example (NdCeFeB) you will be able to find at most 5 groups:
- `sample` containing basic sample info such as layer structure, deposition date, operator, ect...
- `NdCeFeB_1-1_EDX` containing all EDX scans at each wafer position.
- `NdCeFeB_2-5_ESRF` containing all XRD scans at each wafer position.
- `NdCeFeB_2-5_MOKE` containing all MOKE scans at each wafer position.

These group will be reviewed in more details using the reader itself, but feel free to explore the dataset on your own and please if you have any suggestions.

First, you need to install all the required packages using the following command:

In [ ]:
# This step is optional and not needed if all packages are already present 
#!source .venv/bin/activate
#!pip install -e .

### **3. Using the hdf5 reader from Institut Néel**

##### **3.1 Default xarray full dataset**

A built-in module has been written to parse the full hdf5 file, it will read all the data along with the attributs associated to each dataset and convert it to an xarray.Dataset object. 

From the xarrays you will be able to manipulate and plot the data as your convenience.

First, you can import the `read_hdf5` functions along the `pathlib` python library to use path that are OS independant.

In [1]:
import neel_data_vis.readers.read_hdf5 as read_hdf5
import pathlib

Then you simply need to specify the path to your HDF5 datafile using the following command: 

In [2]:
HDF5_path = pathlib.Path("/Users/williamrigaut/Desktop/combinatorials/combinatorials-data/HDF5/NdCeFeB/NdCeFeB_2-7.hdf5")

The package `read_hdf5` includes a function `get_full_dataset()` that will read inside the HDF5 file to retrieve results from <b>EDX</b>, <b>MOKE</b> and <b>XRD</b> for every position using an xarray.

You can call it by using the following:


In [3]:
data = read_hdf5.get_full_dataset(HDF5_path)
data # Displays the xarray Dataset

Data type PROFIL not found in HDF5 file.


<xarray.Dataset> Size: 37kB
Dimensions:                                               (y: 17, x: 17)
Coordinates:
  * y                                                     (y) float64 136B -4...
  * x                                                     (x) float64 136B -4...
Data variables: (12/16)
    Ce Composition                                        (y, x) float64 2kB ...
    Fe Composition                                        (y, x) float64 2kB ...
    Nd Composition                                        (y, x) float64 2kB ...
    coercivity_dmdh                                       (y, x) float64 2kB ...
    coercivity_m0                                         (y, x) float64 2kB ...
    intercept_field                                       (y, x) float64 2kB ...
    ...                                                    ...
    XRD_NdCeFeB_2-7 Ceriumoxidealpha Lattice Parameter C  (y, x) float64 2kB ...
    XRD_NdCeFeB_2-7 Ironalpha Phase Fraction              (y, x) float64 2kB ...
    XRD_NdCeFeB_2-7 Ironalpha Lattice Parameter A         (y, x) float64 2kB ...
    XRD_NdCeFeB_2-7 Nd2Fe14B Phase Fraction               (y, x) float64 2kB ...
    XRD_NdCeFeB_2-7 Nd2Fe14B Lattice Parameter A          (y, x) float64 2kB ...
    XRD_NdCeFeB_2-7 Nd2Fe14B Lattice Parameter C          (y, x) float64 2kB ...

 An interactive widget will allow you to see the content of the dataset. Currently this function returns an xarray.Dataset object with Coordinates (x,y) and the Data variables are :
 
 - `A_i Composition` (in at.%) is the compofrsition of A_i from EDX quantification.
 
 - `coercivity_dmdh` is the derivate coercivity dM/dH.
 - `coercivity_m0` (in T) is the coercivity when M = 0.
 - `intercept_field` (in T) is the intercept field.
 - `max_kerr_rotation` (in °) is the maximum signal from MOKE.
 - `reflectivity` (in V) is the MOKE reflectivity.
 - `P_i Phase Fraction` (in wt.%) is the weight phase fraction of P_i from XRD.
 - `(P_i)_j Lattice Parameter j` (in nm) is the lattice parameter j of the phase P_i obtained from XRD.
 - `measured_height` (in nm) is the thickness of the film.

 Since the coordinate matrix is a 80x80mm square and the wafer sample is shape like a disk, some points are actually outside the wafer and does not have any values associated and are set to np.nan values instead. Also, if multiple datasets for a single measurement technique is present, the data variable will be preceed by the dataset name. <span style="color: red;">If the hdf5 dataset contains multiple entries for the same measurement technique, a prefix with the dataset name will be added to the data variable name.</span>

If you are only interested in one data variable, for example `Coercivity`, you can simply select this data with:

In [4]:
coercivity = data["coercivity_m0"] # Data variables can be selected using the dictionary syntax in python
coercivity

<xarray.DataArray 'coercivity_m0' (y: 17, x: 17)> Size: 2kB
array([[       nan,        nan,        nan,        nan,        nan,
               nan, 0.69965305, 0.70649816, 0.69431504, 0.67916687,
        0.6414323 ,        nan,        nan,        nan,        nan,
               nan,        nan],
       [       nan,        nan,        nan,        nan, 0.7533019 ,
        0.8300083 , 0.81406389, 0.8356194 , 0.8537124 , 0.78731245,
        0.74788779, 0.67721641, 0.59555166,        nan,        nan,
               nan,        nan],
       [       nan,        nan,        nan, 0.80544614, 0.87498949,
        0.94460942, 1.01424921, 1.09287595, 1.06130136, 0.99110801,
        0.94623359, 0.70200093, 0.64792245, 0.56444716,        nan,
               nan,        nan],
       [       nan,        nan, 0.81293123, 0.92220567, 1.02803269,
        1.14114286, 1.2226827 , 1.26930991, 1.16160464, 1.20652779,
        1.05892928, 0.83714701, 0.74724782, 0.58872443, 0.50411627,
               nan,        nan],
       [       nan, 0.81592056, 0.87739012, 1.07464049, 1.20745967,
        1.31896897, 1.40422123, 1.41605322, 1.3414418 , 1.32049876,
        1.11769394, 1.01688621, 0.84509577, 0.70267961, 0.55512137,
        0.4646553 ,        nan],
...
       [       nan, 0.76362077, 0.83304236, 0.96174372, 0.99016773,
        0.9878612 , 1.05472562, 1.03144289, 0.92947883, 0.8957801 ,
        0.82783426, 0.71949423, 0.62876851, 0.50597844, 0.37675733,
        0.33016461,        nan],
       [       nan,        nan, 0.72854343, 0.76347288, 0.83379111,
        0.90261167, 0.87713424, 0.89102074, 0.89708291, 0.8032804 ,
        0.73471599, 0.60940636, 0.50380274, 0.44368801, 0.39706557,
               nan,        nan],
       [       nan,        nan,        nan, 0.65744362, 0.7299107 ,
        0.72430086, 0.72910374, 0.76677744, 0.74158778, 0.6914039 ,
        0.61671838, 0.50600954, 0.45370482, 0.34886057,        nan,
               nan,        nan],
       [       nan,        nan,        nan,        nan, 0.61017123,
        0.61114328, 0.57907165, 0.61067153, 0.56495717, 0.48333166,
        0.42150857, 0.35919667, 0.35378899,        nan,        nan,
               nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan, 0.48440695, 0.49987449, 0.41930401, 0.35421203,
        0.34091646,        nan,        nan,        nan,        nan,
               nan,        nan]])
Coordinates:
  * y        (y) float64 136B -40.0 -35.0 -30.0 -25.0 ... 25.0 30.0 35.0 40.0
  * x        (x) float64 136B -40.0 -35.0 -30.0 -25.0 ... 25.0 30.0 35.0 40.0
Attributes:
    units:    T

- To better visualize the data, you can plot the data in a 2D heatmap using `plot()` directly on the xarray. However for better consistancy with DaHU we choose to use the `plotly` library instead.

In [5]:
# Heatmap plot
import neel_data_vis.readers.plotting as plt

coercivity = data["coercivity_m0"] # Getting all the coercivity values
graph_title = HDF5_path.stem + "_MOKE"
coloraxe_title = "µ\u2080Hc (T)"
range_color = (float(coercivity.min().values), float(coercivity.max().values))

fig = plt.plot_heatmap(coercivity, graph_title=graph_title, coloraxe_title=coloraxe_title, colorscale="rainbow", range_color=range_color)

fig.show()

You can also select subsets with the `sel()` function by specifiying in argument which values of x and y you need. For more advanced xarray operations you can directly check https://docs.xarray.dev/en/stable/getting-started-guide/quick-overview.html

The following examples include a line selection for Nd composition.

In [9]:
# Line x=0 selected for Nd composition
import neel_data_vis.readers.plotting as plt

composition_Nd = data["Nd Composition"] # Nd composition data variable
line_composition = composition_Nd.sel(y=0) # Selecting the line y=0
coordinates = line_composition.coords["x"] # Change .coords["x"] to .coords["y"] if you plot at x fixed

graph_title = "Nd Composition at line y=0"
yaxis_title = "Nd Composition (at %)"

fig = plt.plot_1d(x=coordinates, y=line_composition, y_range=(6,12), graph_title=graph_title, xaxis_title="X (mm)", yaxis_title=yaxis_title)

fig.show()

Here is another heatmap example with the lattice parameter C for NdFeB :

In [10]:
# Lattice parameter C for NdFeB
import neel_data_vis.readers.plotting as plt

lattice_parameter_C_NdFeB = data["XRD_NdCeFeB_2-7 Nd2Fe14B Lattice Parameter C"] #Lattice parameter C for Nd2Fe14B phase, here we need to add the dataset name XRD_NdCeFeB_2-7 as prefix because multiple datasets for XRD data are present

graph_title = "Lattice parameter C for NdFeB phase"
coloraxe_title = "C (nm)"
range_color = (float(lattice_parameter_C_NdFeB.min().values), float(lattice_parameter_C_NdFeB.max().values)) # To get min and max values

fig = plt.plot_heatmap(lattice_parameter_C_NdFeB, graph_title=graph_title, coloraxe_title=coloraxe_title, colorscale="rainbow", range_color=range_color, precision=4)

fig.show()

You can then associate multiple datasets to plot for example lattice parameter C vs Ce over total RE content and coercivity in a 1D plot. For this I recommand using the `pandas` module as well after reshaping our data:

In [12]:
# More manipulations
import plotly.graph_objects as go
import neel_data_vis.readers.plotting as plt

# Setting values for default NdFeB and CeFeB lattice parameters
DEFAULT_CE_CONTENT = [100, 0]
DEFAULT_LATTICE_PARAMETER_C = [1.210, 1.221]

# If we want to do operations on the data, we reshape it to a 1D array first
ce_content = data["Ce Composition"].values.reshape(-1)
fe_content = data["Fe Composition"].values.reshape(-1)
coercivity = data["coercivity_m0"].values.reshape(-1)
lattice_c = data["XRD_NdCeFeB_2-7 Nd2Fe14B Lattice Parameter C"].values.reshape(-1)

# We can also include a mask to remove outliers, for example here we only want coercivity values from 0.3 to 1.6 T
mask_min = 0.1; mask_max = 1.6

# We create the scatter plot with the amount of Ce/RE in x, the lattice parameter C in y and the coercivity in color
fig = plt.plot_scatter(
    x=100*ce_content/(100-fe_content), y=lattice_c, color=coercivity, mask=(mask_min, mask_max),
    x_label="Ce/RE content (at.%)", y_label="Lattice Parameter C (nm)", color_label="µ\u2080Hc (T)",
    x_range=(20, 55), y_range=(1.214, 1.219), graph_title="Lattice Parameter C and coercivity as a function of Ce content", colorscale="plasma")

# We add the blue line for the Veigar's law with lattice parameters for NdFeB and CeFeB found in litterature
fig.add_trace(go.Scatter(x=DEFAULT_CE_CONTENT, y=DEFAULT_LATTICE_PARAMETER_C, marker={'color': 'blue'}, showlegend=False))

fig.show()

Another example is a ternary plot for Nd, Ce and Fe composition vs coercivity. Here we are also using `pandas` so the data is reshaped to a 1D array first. You can then zoom on the plot with plotly using the zoom button on the top right corner of the figure.

In [15]:
# Ternary plot
import neel_data_vis.readers.plotting as plt

Nd = data["Nd Composition"]
Ce = data["Ce Composition"]
Fe = data["Fe Composition"]
coercivity = data["coercivity_m0"]
lattice_c = data["XRD_NdCeFeB_2-7 Nd2Fe14B Lattice Parameter C"]
lattice_a = data["XRD_NdCeFeB_2-7 Nd2Fe14B Lattice Parameter A"]

mask_min = 0.39; mask_max = 1.6 # We can also include a mask to remove outliers

# Here we plot coercivity as a function of composition but you can replace it with any other variable such as lattice_c, give it a try !
fig = plt.plot_ternary(a=Nd, b=Ce, c=Fe, color=coercivity, mask=(mask_min, mask_max),
    a_label="Nd (at.%)", b_label="Ce (at.%)", c_label="Fe (at.%)", color_label="Coercivity (T)", colorscale="rainbow")

fig.show() # You can use the zoom from the plot to get a better view of the data

##### **3.2 Getting xarray datasets containing measurement data**

If you need to work with the measurement readout instead, the function `get_measurement_data()` will return an xarray Dataset object containing the scans for each measurement type.

In [18]:
import neel_data_vis.readers.read_hdf5 as read_hdf5
import pathlib

HDF5_path = pathlib.Path("/Users/williamrigaut/Desktop/combinatorials/combinatorials-data/HDF5/NdCeFeB/NdCeFeB_2-7.hdf5")

edx_scans = read_hdf5.get_measurement_data(HDF5_path, datatype="edx") # We specify the datatype we want
moke_scans = read_hdf5.get_measurement_data(HDF5_path, datatype="moke")
#xrd_scans = read_hdf5.get_measurement_data(HDF5_path, datatype="xrd")

# If multiple measurements are present, we have to specify the meas_key.
# But if no key is specified, the function will return a list of available meas_key, for example:
xrd_scans = read_hdf5.get_measurement_data(HDF5_path, datatype="xrd", meas_key="XRD_NdCeFeB_2-7")

Reading edx
Reading moke
Reading xrd


Before plotting, you need to select which dataset you want to plot as a function of another one, as an example for XRD :

In [19]:
# For 1D XRD pattern at x=0, y=0
import neel_data_vis.readers.plotting as plt

xrd_x0y0 = xrd_scans.sel(x=-20, y=-35) # We select the XRD pattern at x=0, y=0

# Then we extract the angle and the intensity, we can use xrd_x0y0.keys() to see the available keys in the Xarray
tth = xrd_x0y0["tth"]
q = xrd_x0y0["q"] # q is in Å^-1
counts = xrd_x0y0["counts"] 

graph_title = "XRD pattern at position x=0, y=0"

fig = plt.plot_1d(x=tth, y=counts, xaxis_title="Two Theta (°)", yaxis_title="Intensity (a.u.)", graph_title=graph_title)

fig.show()

One beautiful way to look at XRD patterns is with a waterfall plot. Here you select a line (x=0 as an example) and you plot an image with the intensity on the color scale, the angle on the x axis and the position on the y axis.

In [21]:
# Waterfall pattern
import neel_data_vis.readers.plotting as plt
import numpy as np

tth = xrd_scans.sel(x=0, y=0)["tth"]
counts = xrd_scans.sel(x=0)["counts"] # Here we only select all the XRD patterns at the line x=0
position_counts = xrd_scans["counts"].sel(x=0).coords["y"] # Change to "y" if you plot at x fixed
range_color = (0, float(counts.max()))

fig = plt.plot_waterfall(counts, x=tth, y=position_counts,
    graph_title="NdCeFeB_2-7 XRD pattern at line x=0", coloraxe_title="Intensity (a.u.)", colorscale="plasma", range_color=range_color)
fig.update_layout(xaxis_title="Two Theta (°)")
fig.show()


### **4. Data export to different formats such as image, powerpoint or csv**

Here are some utility functions to export data into different formats. Common use cases are to export the data as a png file or as a csv file.
The code is not encapsulated in the  `read_hdf5` module, so you can adjust it to your convenience.

##### **4.1 Export all heatmaps as a png file from a HDF5 file**

The following code allows you to export all heatmaps as png files from a HDF5 file.

In [22]:
import neel_data_vis.readers.read_hdf5 as read_hdf5
import plotly.express as px
import pathlib
from PIL import Image

HDF5_path = pathlib.Path("/Users/williamrigaut/Desktop/combinatorials/combinatorials-data/HDF5/NdCeFeB/NdCeFeB_2-7.hdf5")
data = read_hdf5.get_full_dataset(HDF5_path)

# Creating new folders for the png files
!mkdir plots/
!mkdir plots/{HDF5_path.stem}

# Set this as your convenience
colorscale = "plasma"
xaxis_title = "X (mm)"
yaxis_title = "Y (mm)"
xyaxis_tickfont_sizes = 22
xyaxis_title_font_sizes = 20

# Going through the data and changing the graph title, coloraxe_title, precision and range_color depending on the datatype
for key in data.keys():
    current_data = data[key]
    min_data = float((current_data.min()).values)
    max_data = float((current_data.max()).values)

    if "Composition" in key:
        graph_title = HDF5_path.stem + "_EDX " + key
        coloraxe_title = current_data.attrs["units"]
        precision = 1
        range_color = (round(min_data, precision), round(max_data, precision))
    
    elif "coercivity_m0" in key:
        graph_title = HDF5_path.stem + "_MOKE " + key
        coloraxe_title = f"µ\u2080Hc ({current_data.attrs["units"]})"
        precision = 1
        range_color = (round(min_data, precision), round(max_data, precision))
    
    elif "max_kerr_signal" in key:
        graph_title = HDF5_path.stem + "_MOKE " + key
        coloraxe_title = current_data.attrs["units"]
        precision = 2
        range_color = (round(min_data, precision), round(max_data, precision))
    
    elif "Phase Fraction" in key:
        graph_title = HDF5_path.stem + "_XRD " + key
        coloraxe_title = current_data.attrs["units"]
        precision = 0
        range_color = (round(min_data*100, precision), round(max_data*100, precision))
        current_data = current_data*100
    
    elif "Lattice Parameter" in key:
        graph_title = HDF5_path.stem + "_XRD " + key
        coloraxe_title = current_data.attrs["units"]
        precision = 4
        range_color = (round(min_data, precision), round(max_data, precision))
    else :
        continue
    
    print(f"Creating Heatmap for: {key}")
    fig = plt.plot_heatmap(current_data, graph_title=graph_title, coloraxe_title=coloraxe_title, colorscale=colorscale, range_color=range_color, precision=precision)
    fig.update_layout(coloraxis_colorbar_x=0.85, title_x=0.5)

    #Saving and cropping the image
    image_path = f"plots/{HDF5_path.stem}/{key}.jpeg"
    fig.write_image(image_path, scale=2)
    img = Image.open(image_path)
    width, height = img.size
    img = img.crop((170, 20, width-70, height-40))
    img.save(image_path)

Data type PROFIL not found in HDF5 file.
mkdir: plots/: File exists
Creating Heatmap for: Ce Composition
Creating Heatmap for: Fe Composition
Creating Heatmap for: Nd Composition
Creating Heatmap for: coercivity_m0
Creating Heatmap for: max_kerr_signal
Creating Heatmap for: XRD_NdCeFeB_2-7 Ceriumoxidealpha Phase Fraction
Creating Heatmap for: XRD_NdCeFeB_2-7 Ceriumoxidealpha Lattice Parameter A
Creating Heatmap for: XRD_NdCeFeB_2-7 Ceriumoxidealpha Lattice Parameter C
Creating Heatmap for: XRD_NdCeFeB_2-7 Ironalpha Phase Fraction
Creating Heatmap for: XRD_NdCeFeB_2-7 Ironalpha Lattice Parameter A
Creating Heatmap for: XRD_NdCeFeB_2-7 Nd2Fe14B Phase Fraction
Creating Heatmap for: XRD_NdCeFeB_2-7 Nd2Fe14B Lattice Parameter A
Creating Heatmap for: XRD_NdCeFeB_2-7 Nd2Fe14B Lattice Parameter C


##### **4.2 Generate a powerpoint from the png files created**

From the images created in the previous step, you can generate a powerpoint with the following code:

In [23]:
from pptx import Presentation
from pptx.util import Inches
import glob

# Defining the path to the images
image_folder_path = '/Users/williamrigaut/Desktop/Notebooks/Advanced_Data_Visualization/plots/NdFeB_100mm'

# Creating the presentation with default size
prs = Presentation()
prs.slide_width = Inches(13.33)
prs.slide_height = Inches(7.5)
blank_slide_layout = prs.slide_layouts[6] # Layout 6 is a blank slide

# Figure size in the slide (fixed to fit 6 per slide)
height = Inches(3.5)

# Adding the images in the slides in order (EDX -> MOKE -> XRD)
all_images = [f"{image_folder_path}/*Composition*.jpeg", [f"{image_folder_path}/*coercivity_m0*.jpeg", f"{image_folder_path}/*max_kerr_signal*.jpeg"], f"{image_folder_path}/*Phase Fraction*.jpeg", f"{image_folder_path}/*Lattice Parameter*.jpeg"]

for img_path in all_images:
    # To account for the order
    if isinstance(img_path, list):
        glob_img_path = glob.glob(img_path[0]) + glob.glob(img_path[1])
    elif isinstance(img_path, str):
        glob_img_path = glob.glob(img_path)

    # Adding the images in the slides
    j = 0 # Slide counter, used to reset the image position when we go to the next slide
    for i, img in enumerate(sorted(glob_img_path)):
        if i%6 == 0:
            slide = prs.slides.add_slide(blank_slide_layout) # Adding a new slide
            j += 1
        # Defining the position of the image on the slide
        top = Inches(0.1 + ((i-6*(j-1))//3*3.65))
        left = Inches(4.5*(i%3))
        pic = slide.shapes.add_picture(img, left, top, height=height) # Adding the image

# Saving the presentation
save_path = f'plots/{image_folder_path.split("/")[-1]}/{image_folder_path.split("/")[-1]}.pptx'
prs.save(save_path)
print(f"Presentation saved at {save_path}")

Presentation saved at plots/NdFeB_100mm/NdFeB_100mm.pptx


##### **4.3 Exporting to csv using mammos-entities**

Here we show an example of exporting the data to a csv file using MaMMos-entities which takes care of (magnetic-) units (https://pypi.org/project/mammos-entity/).

In [24]:
import neel_data_vis.readers.read_hdf5 as read_hdf5
import pathlib
import numpy as np

import mammos_units as u
import mammos_entity as me

In [25]:
def create_lists_for_me_from_hdf5(HDF5_path, phase_name="Ceriumoxidealpha Phase Fraction", meas_key=None):
    """
    Extract x and y coordinates and the phase given by the phase_name from a NEEL HDF5 file.
    Returns three lists (x_pos, y_pos, phase_fraction) with one entry per measurement.
    Use these lists for exporting with mammos_entity; numeric values keep their original units (e.g., mm, Wt.%).
    """

    x_pos = []
    y_pos = []
    phase_fraction = []

    # We read the HDF5 file to get the phase fraction data
    print("Reading HDF5 file :", HDF5_path.stem)
    data = read_hdf5.get_full_dataset(HDF5_path, exclude_wafer_edges=False)

    # Phase Fraction" is not in the dataset
    if meas_key is not None:
        phase_name = f"{meas_key} {phase_name}"
    if phase_name not in data.keys():
        print(f"'{phase_name}' not found in dataset. Here is the list of available keys:")
        for key in data.keys():
            print(key)
        return x_pos, y_pos, None
    else:
        print(f"Found '{phase_name}' in dataset.")
        if meas_key is None:
            xrd_scans = read_hdf5.get_measurement_data(HDF5_path, datatype="xrd")
        else:
            xrd_scans = read_hdf5.get_measurement_data(HDF5_path, datatype="xrd", meas_key=meas_key)

        x_vals = np.array(xrd_scans.x)
        y_vals = np.array(xrd_scans.y)
        
        # We fill values in lists for mammos_entity export
        for m in range(0, len(x_vals)):
            for n in range(0, len(y_vals)):
                x_pos.append(xrd_scans.x[m].values)
                y_pos.append(xrd_scans.y[n].values)

                val = data[phase_name][m, n].values
                phase_fraction.append(val)
        return x_pos, y_pos, phase_fraction


# Setting the hdf5 filepath
HDF5_path = pathlib.Path("/Users/williamrigaut/Desktop/combinatorials/combinatorials-data/HDF5/NdCeFeB/NdCeFeB_2-7.hdf5")
#HDF5_path = pathlib.Path("/home/hortschitz/Documents/Nextcloud_diss/Projekte/2024 MaMMOS/XRD data extraction from NEEL data/datasets/NdCeFeB_2-7_20251021.hdf5")

# Use pathlib to get filename and stem
filename_with_extension = HDF5_path.name
filename_without_extension = HDF5_path.stem

# extracting the lists of data from the hdf5 file to export as mammos entities
x_pos, y_pos, ceriumoxidealpha_phase_fraction = create_lists_for_me_from_hdf5(HDF5_path,
                                                                              phase_name="Ceriumoxidealpha Phase Fraction",
                                                                              meas_key="XRD_NdCeFeB_2-7")

# %% Export data as mammos entities
x_pos_me = x_pos * u.mm
y_pos_me = y_pos * u.mm

# data["Ceriumoxidealpha Phase Fraction"].units gives 'Wt.%'
Wt_pct = u.def_unit("Wt_pct", u.kg / u.kg * u.percent)
ceriumoxidealpha_phase_fraction_me = ceriumoxidealpha_phase_fraction * Wt_pct

# Use pathlib to construct the output path
current_file_path = HDF5_path.parent.resolve()
output_csv_mammos = str(current_file_path / (filename_without_extension + "_ceriumOxide_phase_fractions_mammos_entities.csv"))

me.io.entities_to_file(
    output_csv_mammos,
    "CeriumOxide phase fractions"
    + "from "
    + filename_with_extension
    + "\n\n"
    + " Using mammos_units to define Wt.%:"
    + " 'Wt_pct = u.def_unit('Wt_pct', u.kg/u.kg*u.percent)'",
    x_pos=x_pos_me,
    y_pos=y_pos_me,
    ceriumoxidealpha_phase_fraction=ceriumoxidealpha_phase_fraction_me,
)
print(f"Mammos entities written to: {output_csv_mammos}")


Reading HDF5 file : NdCeFeB_2-7
Data type PROFIL not found in HDF5 file.
Found 'XRD_NdCeFeB_2-7 Ceriumoxidealpha Phase Fraction' in dataset.
Reading xrd
Mammos entities written to: /Users/williamrigaut/Desktop/combinatorials/combinatorials-data/HDF5/NdCeFeB/NdCeFeB_2-7_ceriumOxide_phase_fractions_mammos_entities.csv


### **Appendix A. CHADA Tables and Diagrams for High Throughput Experiments at Néel Institut**

#### **A.1 CHADA Diagrams**

CHADA Diagrams are used to describe the flowchart of an experiment with a simple skematic. Each step is described with a short explanation and a color:

- RED: Indicates what type of sample was used to start the experiment.

- BLUE: Shows what needs to be done during the experiment to obtain the measured data.

- DARK GREEN: Raw data measured directly from the experiment.

- LIGHT GREEN: Treated data, usually extracted for the raw data using a software or a python code.

- DARK BLUE CONTOUR: Final treated data, is the content that you obtain from the `get_full_dataset()` command.

##### **CHADA Diagram for EDX (click left to display):**

![EDX Diagram](examples/EDX.svg)

##### **CHADA Diagram for MOKE:**

![MOKE Diagram](examples/MOKE.svg)

##### **CHADA Diagram for XRD:**

![XRD Diagram](examples/XRD.svg)

#### **A.2 CHADA Tables**
CHADA Tables are used to describe in details each experiment with a longer description that diagrams. Each important point about the experiment is described in a few lines. The color code is the same as for CHADA Tables.

##### **CHADA Table for EDX:**

![EDX Table](examples/CHADA_TABLES_EDX.svg)

##### **CHADA Table for MOKE:**

![MOKE Table](examples/CHADA_TABLES_MOKE.svg)

##### **CHADA Table for XRD:**

![XRD Table](examples/CHADA_TABLES_XRD.svg)

### **@end-of-notebook**